In [ ]:
!pip install requests
!pip install lxml

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# API Endpoint (Base URL)
base_url_list = "http://www.khs.go.kr/cha/SearchKindOpenapiList.do"
base_url_detail = "http://www.khs.go.kr/cha/SearchKindOpenapiDt.do"

kind_codes = ["11","12","13","14","15","16","17","18","21","22","23","24","25"]
#city_codes = ["11","21","22","23","24","25","26","45","31","32","33","34","35","36","37","38","50"]
city_codes = ["50"]
all_data = []

for kdcd in kind_codes:
  for ctcd in city_codes:
    page_idx = 1
    while True:
      params = {
        "ccbaKdcd": kdcd,
        "ccbaCtcd": ctcd,
        "pageUnit": 100,
        "pageIndex": page_idx
      }
      response = requests.get(base_url_list, params=params)
      if response.status_code != 200:
        print(f"목록 API 에러 code: {response.status_code}")
        break

      root = ET.fromstring(response.text)
      items = root.findall(".//item")

      if not items:
        break  # 더 이상 데이터 없으면 종료

      for item in items:
        # 목록 API에서 기본 정보 추출
        name = item.findtext("ccbaMnm1")
        kind = item.findtext("ccmaName")
        location = item.findtext("ccbaCtcdNm", "") + " " + item.findtext("ccsiName", "")
        lon = item.findtext("longitude")
        lat = item.findtext("latitude")

        # 상세조회에 필요한 파라미터(관리번호, 종목코드, 시도코드)
        asno = item.findtext("ccbaAsno")
        kdcd_val = item.findtext("ccbaKdcd")  # 혹은 kdcd
        ctcd_val = item.findtext("ccbaCtcd")  # 혹은 ctcd

        # 상세 정보 API 호출
        detail_params = {
          "ccbaAsno": asno,
          "ccbaKdcd": kdcd_val,
          "ccbaCtcd": ctcd_val
        }
        detail_res = requests.get(base_url_detail, params=detail_params)
        detail_content = ""

        if detail_res.status_code == 200:
          detail_root = ET.fromstring(detail_res.text)
          detail_item = detail_root.find(".//item")
          if detail_item is not None:
            detail_content = detail_item.findtext("content", "")
            detail_address = detail_item.findtext("ccbaLcad", "")
        else:
          print(f"상세 API 에러 code: {detail_res.status_code}, asno={asno}")

        # 최종 레코드
        all_data.append({
          # "종목코드": kdcd_val,
          # "시도코드": ctcd_val,
          # "관리번호": asno,
          "문화재명": name,
          "분류":kind,
          "소재지": location.strip(),
          "상세주소": detail_address,
          "위도": lon,
          "경도": lat,
          "상세설명": detail_content
        })

      page_idx += 1


print(f"{len(all_data)}")

df = pd.DataFrame(all_data)
df.head()


417


,문화재명,분류,소재지,상세주소,위도,경도,상세설명
0,제주 관덕정,보물,제주 제주시,\n\t\t\t\n\t \n\t ...,126.521489474308,33.5133609436166,제주시내 중심부에 위치한 제주 관덕정은 제주에서 가장 오래된 건물 중의 하나다. \...
1,김정희 종가 유물 일괄,보물,제주 서귀포시,\n\t\t\t\n\t \n\t ...,126.560076,33.2541205,추사 김정희 고택의 전래 유물로 추정되는 월성위 김한신 관련유물(10점)과 『신해년...
2,안중근의사 유묵 - 천여불수반수기앙이,보물,제주 제주시,\n\t\t\t\n\t \n\t 제주특별자치도 ...,0,0,「안중근의사유묵-천여불수반수기앙이(安重根義士遺墨-天與不受反受其殃耳)」는 안중근 의사...
3,제주 불탑사 오층석탑,보물,제주 제주시,\n\t\t\t\n\t \n\t ...,126.598544211899,33.5293780667049,"원당사의 옛 터에 세워져 있는 석탑이다. 원당사는 조선 중기에 폐지되었고, 1950..."
4,동여비고,보물,제주 제주시,\n\t\t\t\n\t \n\t ...,0,0,조선 후기 전국을 그린 지도책으로 32여종의 지도를 그린 60면으로 구성되어 있다....


In [ ]:
df["위도"] = pd.to_numeric(df["위도"], errors="coerce")
df["경도"] = pd.to_numeric(df["경도"], errors="coerce")

df_not_zero = df[
    (df["위도"] != 0) &
    (df["경도"] != 0) &
    df["위도"].notna() &
    df["경도"].notna()
]

print(df_not_zero.shape)
df_not_zero.head()

(333, 7)


,문화재명,분류,소재지,상세주소,위도,경도,상세설명
0,제주 관덕정,보물,제주 제주시,\n\t\t\t\n\t \n\t ...,126.521489,33.513361,제주시내 중심부에 위치한 제주 관덕정은 제주에서 가장 오래된 건물 중의 하나다. \...
1,김정희 종가 유물 일괄,보물,제주 서귀포시,\n\t\t\t\n\t \n\t ...,126.560076,33.254120,추사 김정희 고택의 전래 유물로 추정되는 월성위 김한신 관련유물(10점)과 『신해년...
3,제주 불탑사 오층석탑,보물,제주 제주시,\n\t\t\t\n\t \n\t ...,126.598544,33.529378,"원당사의 옛 터에 세워져 있는 석탑이다. 원당사는 조선 중기에 폐지되었고, 1950..."
5,제주향교 대성전,보물,제주 제주시,\n\t\t\t\n\t \n\t ...,126.515618,33.511552,제주향교는 태조 3년(1394년) 관덕정에서 동쪽으로 1리 정도 떨어진 가락천의 서...
7,제주 삼성혈,사적,제주 제주시,\n\t\t\t\n\t \n\t ...,126.528872,33.504301,삼성혈은 제주도의 고씨·양씨·부씨의 시조가 솟아났다는 3개의 구멍을 말한다.\n 3...


In [ ]:
df_not_zero.to_csv("jeju_heritage.csv", index=False, encoding="utf-8-sig")